In [ ]:
import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
from finlab import backtest
import pandas as pd

# 策略需用資料
收盤價 = data.get("price:收盤價")
long_ma_pattern = (收盤價 > 收盤價.average(50)) & (收盤價 > 收盤價.average(150)) & (收盤價 > 收盤價.average(200))
RS = ((收盤價/收盤價.shift(20)) / (收盤價['0050']/收盤價['0050'].shift(20)))
price_pct = 收盤價.pct_change(10)
去年同月增減 = data.get("monthly_revenue:去年同月增減(%)")
rev_rise_nsatisfy = 去年同月增減.rank(pct=True,axis=1)>0.8
data.universe_stocks = {}
score = data.get('etl:finlab_tw_stock_market_ind')['score']
inventory = data.get("inventory")

#散戶比例
small_inv = dataframe.FinlabDataFrame(inventory[(inventory.持股分級.astype(int) <= 8)].reset_index().groupby(["date", "stock_id"]).agg({"占集保庫存數比例": "sum"}).reset_index().pivot("date", "stock_id")["占集保庫存數比例"])

#為指標打分
def qcut_feature(data_name='RSrank', q_range=10, ascending=True):
    import numpy as np

    rank_df = ((data_name).rank(axis=1, pct=True, ascending=ascending).mul(q_range).add(1).apply(np.floor).clip(0,q_range))
    return rank_df
RSrank = qcut_feature(data_name=RS)

cond1 = long_ma_pattern 
cond2 = small_inv.fall().sustain(8,3)  
cond3 = ((收盤價.average(200)- (收盤價.average(200).shift(20))) > 0)
cond4 = 收盤價 >= 1.25*(收盤價.rolling(260).min())
cond5 = 收盤價 >= 0.75*(收盤價.rolling(260).max())
cond6 = RSrank >= 9  
cond7 = price_pct.is_largest(10) & rev_rise_nsatisfy

cond_all = cond1 & cond2 &cond3 &cond4 & cond5 & cond6 & cond7

#出場
sma10 = 收盤價.average(10)
sell = (sma10 < sma10.shift()) 
entries = cond_all
score_df = score >= 4
entries *= score_df

entries = entries[entries.columns[~entries.columns.str[-1].isin(['L', 'B', 'C'])]]
exits = sell 

position = entries.hold_until(exits, nstocks_limit=5 ,stop_loss = 0.15, take_profit=0.6 ,rank=RSrank )

report = sim(position, resample="w", position_limit=0.333, stop_loss=0.15 ,name='超級績效使用RS指標與營收與散戶較少加大盤濾網', mae_mfe_window=7, trade_at_price='close' )